In [ ]:
!kaggle competitions download -c march-machine-learning-mania-2025

In [2]:
import numpy as np
import pandas as pd
import warnings
import missingno
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)


In [3]:
# load data
data_dir = r"C:\Users\user\MachineLearnings\NCCA\march-machine-learning-mania-2025"
M_regular_results = pd.read_csv(f"{data_dir}\MRegularSeasonDetailedResults.csv")
M_tourney_results = pd.read_csv(f"{data_dir}\MNCAATourneyDetailedResults.csv")
W_regular_results = pd.read_csv(f"{data_dir}\WRegularSeasonDetailedResults.csv")
W_tourney_results = pd.read_csv(f"{data_dir}\WNCAATourneyDetailedResults.csv")
M_seeds = pd.read_csv(f"{data_dir}\MNCAATourneySeeds.csv")
W_seeds = pd.read_csv(f"{data_dir}\WNCAATourneySeeds.csv")

In [4]:
# data 정리
# 남여 합치기
regular_results = pd.concat([M_regular_results, W_regular_results], axis=0) 
tourney_results = pd.concat([M_tourney_results, W_tourney_results], axis=0)
seeds = pd.concat([M_seeds, W_seeds], axis=0)
# 2005년도 이상만 포함
regular_results = regular_results[regular_results["Season"] >= 2005]
tourney_results = tourney_results[tourney_results["Season"] >= 2005]
seeds = seeds[seeds["Season"] >= 2005]
#seed 번호 정수로 정리
seeds['seed'] = seeds["Seed"].apply(lambda x: int(x[1:3]))


In [5]:

def prepare_data(df):
    df = df[["Season", "DayNum", "LTeamID", "LScore", "WTeamID", "WScore", "NumOT",
            "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF",
            "WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]]
    # 연장전 보정
    adjot = (40 + 5*df['NumOT'])/40
    adj_col = ["LScore", "WScore", "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF",
               "WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]
    for col in adj_col:
        df[col] = df[col] / adjot
    # 한경기를 T1, T2 / T2, T1으로 나누기 
    df_swap = df.copy()
    df.columns = [x.replace("W","T1_").replace("L", "T2_") for x in list(df.columns)]
    df_swap.columns = [x.replace("L","T1_").replace("W", "T2_") for x in list(df_swap.columns)]
    output = pd.concat([df, df_swap], axis=0).reset_index(drop=True)
    output["PointDiff"] = output["T1_Score"] - output["T2_Score"]
    output["win"] = np.where(output["PointDiff"] > 0, 1, 0)
    output['gender'] = np.where(output['T1_TeamID']<2000, 1, 0)
    return output

regular_data = prepare_data(regular_results)
tourney_data = prepare_data(tourney_results)
    

In [6]:
# T1, T2의 seed, seed차이 컬럼 붙이기
seeds_T1 = seeds[["Season", "TeamID", "seed"]].copy()
seeds_T2 = seeds[["Season", "TeamID", "seed"]].copy()
seeds_T1.columns = ["Season", "T1_TeamID", "T1_seed"]
seeds_T2.columns = ["Season", "T2_TeamID", "T2_seed"]

tourney_data = tourney_data[["Season", "T1_TeamID", "T2_TeamID", "PointDiff", "win", "gender"]]
tourney_data = pd.merge(tourney_data, seeds_T1, on=["Season", "T1_TeamID"], how="left")
tourney_data = pd.merge(tourney_data, seeds_T2, on=["Season", "T2_TeamID"], how="left")
tourney_data['seed_diff'] = tourney_data['T1_seed'] - tourney_data['T2_seed']

In [7]:
# 슛 성공횟수/시도횟수로 성공률 계산
regular_data["T1_FG%"] = regular_data["T1_FGM"] / regular_data["T1_FGA"]
regular_data["T1_FG3%"] = regular_data["T1_FGM3"] / regular_data["T1_FGA3"]
regular_data["T1_FT%"] = regular_data["T1_FTM"] / regular_data["T1_FTA"]
regular_data["T2_FG%"] = regular_data["T2_FGM"] / regular_data["T2_FGA"]
regular_data["T2_FG3%"] = regular_data["T2_FGM3"] / regular_data["T2_FGA3"]     
regular_data["T2_FT%"] = regular_data["T2_FTM"] / regular_data["T2_FTA"]

In [8]:
boxcols = [
    "T1_Score", "T1_FGM", "T1_FGA", "T1_FG%", "T1_FGM3", "T1_FGA3","T1_FG3%", "T1_FTM", "T1_FTA","T1_FT%",
    "T1_OR", "T1_DR", "T1_Ast", "T1_TO", "T1_Stl", "T1_Blk", "T1_PF",
    "T2_Score", "T2_FGM", "T2_FGA", "T2_FG%", "T2_FGM3", "T2_FGA3","T2_FG3%", "T2_FTM", "T2_FTA","T2_FT%",
    "T2_OR", "T2_DR", "T2_Ast", "T2_TO", "T2_Stl", "T2_Blk", "T2_PF",
    "PointDiff",
]
# box score에 대한 시즌 중간값 구하기
season_median = regular_data.groupby(["Season", "T1_TeamID"])[boxcols].median()

In [ ]:
# 중간값을 T1 팀/ T1의 상대팀들 / T2 팀 / T2의 상대팀들로 만들기
season_median_T1 = season_median.copy()
season_median_T2 = season_median.copy()


season_median_T1.columns = ["T1_avg_" + x.replace("T1_", "").replace("T2_", "opponent_") for x in list(season_median_T1.columns)]


season_median_T2.columns = ["T2_avg_" + x.replace("T1_", "").replace("T2_", "opponent_") for x in list(season_median_T2.columns)]
season_median_T2 = season_median_T2.rename_axis(index={"Season": "Season", "T1_TeamID": "T2_TeamID"}).reset_index()


tourney_data = pd.merge(tourney_data, season_median_T1, on=["Season", "T1_TeamID"], how="left")
tourney_data = pd.merge(tourney_data, season_median_T2, on=["Season", "T2_TeamID"], how="left")

In [ ]:
# team quality 컬럼
# regular data 와 seeds 에 season/TeamID로 되어있는 컬럼 만들기
regular_data["ST1"] = regular_data.apply(lambda t: str(int(t["Season"])) + "/" + str(int(t["T1_TeamID"])), axis=1)
regular_data["ST2"] = regular_data.apply(lambda t: str(int(t["Season"])) + "/" + str(int(t["T2_TeamID"])), axis=1)
seeds_T1["ST1"] = seeds_T1.apply(lambda t: str(int(t["Season"])) + "/" + str(int(t["T1_TeamID"])), axis=1)
seeds_T2["ST2"] = seeds_T2.apply(lambda t: str(int(t["Season"])) + "/" + str(int(t["T2_TeamID"])), axis=1)

# st는 tourney에 진출한 팀들의 lsit, 정규전에서 tourney 진출한 팀을 이긴적 있는 팀도 포함
st = set(seeds_T1["ST1"]) | set(seeds_T2["ST2"])
st = st | set(regular_data.loc[(regular_data["T1_Score"] > regular_data["T2_Score"]) & (regular_data["ST2"].isin(st)), "ST1"])


In [14]:
import statsmodels.api as sm
import tqdm
# team 1과 2 모두 st에 있는 팀만 dt로 
dt= regular_data.loc[regular_data["ST1"].isin(st) | regular_data["ST2"].isin(st)]
dt["T1_TeamID"] = dt["T1_TeamID"].astype(str)
dt["T2_TeamID"] = dt["T2_TeamID"].astype(str)
dt.loc[~dt["ST1"].isin(st), "T1_TeamID"] = "0000"
dt.loc[~dt["ST2"].isin(st), "T2_TeamID"] = "0000"
seasons = sorted(set(seeds["Season"]))

def team_quality(season, gender):
    formula = "PointDiff~-1+T1_TeamID+T2_TeamID"
    glm = sm.GLM.from_formula(formula = formula, 
    data = dt.loc[(dt["Season"] == season) & (dt["gender"] == gender), :],
    family = sm.families.Gaussian(),).fit()
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ["TeamID", "quality"]
    quality["quality"] = quality["quality"]
    quality["Season"] = season
    quality = quality.loc[quality.TeamID.str.contains("T1_")].reset_index(drop=True)
    quality["TeamID"] = quality["TeamID"].apply(lambda x: x[10:14]).astype(int)
    return quality

glm_quality = []
for s in tqdm.tqdm(seasons, unit="season"):
    if s >= 2010:  # min season for women
        glm_quality.append(team_quality(s, 0))
    if s >= 2003:  # min season for men
        glm_quality.append(team_quality(s, 1))

100%|██████████| 20/20 [02:06<00:00,  6.32s/season]


In [15]:
glm_quality = pd.concat(glm_quality).reset_index(drop=True)
glm_quality_T1 = glm_quality.copy()
glm_quality_T2 = glm_quality.copy()
glm_quality_T1.columns = ["T1_TeamID", "T1_quality", "Season"]
glm_quality_T2.columns = ["T2_TeamID", "T2_quality", "Season"]

In [17]:
# 마지막지막 5경기 승률
last_game_day = pd.DataFrame(regular_data.groupby(["ST1"])["DayNum"].max())
last_game_day.columns = ["Last_game_day"]
last_game_add_regular = regular_data.copy()
last_game_add_regular = last_game_add_regular[["ST1", "ST2", "win", "DayNum"]]
last_game_add_regular = last_game_add_regular.merge(last_game_day, left_on = "ST1", right_index = True, how = "left")
last_5_games = last_game_add_regular.groupby("ST1").apply(lambda x: x.nlargest(5, "DayNum")).reset_index(drop=True)

# 최근 5경기의 승률 계산
day_5_ratio = last_5_games.groupby("ST1")["win"].mean().to_frame()
day_5_ratio.columns = ["day_5_ratio"]



In [21]:
game_5_ratio = day_5_ratio.reset_index()
game_5_ratio[["Season", "TeamID"]] = game_5_ratio["ST1"].str.split("/", expand=True)  # "/" 기준으로 분리
game_5_ratio = game_5_ratio.drop(columns=["ST1"])
game_5_ratio["Season"] = game_5_ratio["Season"].astype(int)
game_5_ratio["TeamID"] = game_5_ratio["TeamID"].astype(int)


In [ ]:
# game 5 ratio와 glm quality 합치기

game_5_ratio_T1 = game_5_ratio.copy()
game_5_ratio_T2 = game_5_ratio.copy()
game_5_ratio_T1.columns = ["T1_game_5_ratio", "Season", "T1_TeamID"]
game_5_ratio_T2.columns = ["T2_game_5_ratio", "Season", "T2_TeamID"]
df = pd.merge(tourney_data, game_5_ratio_T1, on=["Season", "T1_TeamID"], how="left")
df = pd.merge(df, game_5_ratio_T2, on=["Season", "T2_TeamID"], how="left")
df['game_5_diff']  = df["T1_game_5_ratio"] - df["T2_game_5_ratio"]




In [25]:
df = pd.merge(df, glm_quality_T1, on=["Season", "T1_TeamID"], how="left")
df = pd.merge(df, glm_quality_T2, on=["Season", "T2_TeamID"], how="left")
df["diff_quality"] = df["T1_quality"] - df["T2_quality"]

In [ ]:
# 시각화 모음
#  1. seed
import plotly.graph_objects as go
import plotly.express as px
win_counts = tourney_data_win.groupby("T1_seed")["win"].count()
loss_counts = tourney_data_loss.groupby("T1_seed")["win"].count()

fig = go.Figure()
fig.add_trace(go.Bar(x=win_counts.index, y=win_counts.values, name="Win"))
fig.add_trace(go.Bar(x=loss_counts.index, y=loss_counts.values, name="Loss"))

fig.update_layout(
    title="Seed별 Win/Loss 횟수",
    xaxis={"title": "T1 Seed", "showticklabels": True, "dtick": 1},
    yaxis={"title": "Count"},
    autosize=False,
    width=800,
    height=600,
    barmode="group"  
)

fig.show()

# 2. quality
tmp = (
    df[["Season", "gender", "T1_seed", "T1_quality"]]
    .drop_duplicates()
    .sort_values("T1_quality")
    .reset_index(drop=True)
)
import matplotlib.pyplot as plt
fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
seaborn.lineplot(tmp.loc[tmp["gender"] == 0, "T1_quality"], color="lightgray", ax=axs[0])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 0) & (tmp.T1_seed == 1), "T1_quality"], color="red", ax=axs[0])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 0) & (tmp.T1_seed == 7), "T1_quality"], color="blue", ax=axs[0])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 0) & (tmp.T1_seed == 16), "T1_quality"], color="green", ax=axs[0])

seaborn.lineplot(tmp.loc[tmp["gender"] == 1, "T1_quality"], color="lightgray", ax=axs[1])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 1) & (tmp.T1_seed == 1), "T1_quality"], color="red", ax=axs[1])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 1) & (tmp.T1_seed == 7), "T1_quality"], color="blue", ax=axs[1])
seaborn.scatterplot(tmp.loc[(tmp["gender"] == 1) & (tmp.T1_seed == 16), "T1_quality"], color="green", ax=axs[1])

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="diff_quality", y="PointDiff", alpha=0.5)
sns.regplot(data=df, x="diff_quality", y="PointDiff", scatter=False, color="red")
plt.axhline(0, color="gray", linestyle="--") 
plt.show()

# 3. 5 game ratio
win_counts = df.groupby("T1_game_5_ratio")["win"].value_counts().unstack(fill_value=0)
fig = go.Figure()
fig.add_trace(go.Bar(
    x=win_counts.index, 
    y=win_counts[0],  
    name=" win=0"
))
fig.add_trace(go.Bar(
    x=win_counts.index, 
    y=win_counts[1],  
    name=" win=1"
))
fig.show()

#  Box scores
def plot(feature):
    num_features = len(feature)

    fig, axes = plt.subplots(nrows=2, ncols=int(num_features/2),figsize=(1.5*num_features, 5))  # 2행 num_features/2열
    axes = axes.flatten()  # Flatten the axes array

    for i, feature in enumerate(feature):
        ax = axes[i]  # subplot 지정
        sns.scatterplot(x=df[feature], y=df["PointDiff"], ax=ax)
        sns.regplot(x=df[feature], y=df["PointDiff"], scatter=False, color="red", ax=ax)  # 회귀선 추가
    
        ax.set_xlabel(feature)
        ax.set_ylabel("PointDiff")
        ax.set_title(f"{feature} vs PointDiff")

    plt.tight_layout()  # 간격 조정
    plt.show()
plot(df_T1)

In [27]:
# feature 선정
features = ['gender',
       'T1_seed', 'T2_seed', 'seed_diff', 'T1_avg_Score', 'T1_avg_FGA',
       'T1_avg_FG%', 'T1_avg_FG3%', 'T1_avg_FGA',
       'T1_avg_DR', 'T1_avg_Ast', 'T1_avg_TO', 'T1_avg_opponent_Score',
       'T1_avg_opponent_FGA',
       'T1_avg_opponent_DR',
       'T1_avg_PointDiff', 'T2_avg_Score',  'T2_avg_FG%', 'T2_avg_FG3%',  'T2_avg_DR',
       'T2_avg_Ast', 'T2_avg_opponent_DR',
       'T2_avg_PointDiff', 'T1_quality', 'T2_quality', 'diff_quality',
       'T1_game_5_ratio', 'T2_game_5_ratio', 'game_5_diff']


In [29]:
# training params 결정
import xgboost as xgb

param = {}
param["objective"] = "reg:squarederror"
param["booster"] = "gbtree"
param["eta"] = 0.01
param["subsample"] = 0.6
param["colsample_bynode"] = 0.8
param["num_parallel_tree"] = 2
param["min_child_weight"] = 4
param["max_depth"] = 4
param["tree_method"] = "hist"
param['grow_policy'] = 'lossguide'
param["max_bin"] = 32

num_rounds = 700

In [30]:
# training
from sklearn.metrics import mean_absolute_error, brier_score_loss

models = {}
oof_mae = []
oof_preds = []
oof_targets = []
oof_ss = []
for oof_season in set(df.Season):
    x_train = df.loc[df["Season"] != oof_season, features].values
    y_train = df.loc[df["Season"] != oof_season, "PointDiff"].values
    x_val = df.loc[df["Season"] == oof_season, features].values
    y_val = df.loc[df["Season"] == oof_season, "PointDiff"].values
    s_val = df.loc[df["Season"] == oof_season, "Season"].values
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dval = xgb.DMatrix(x_val, label=y_val)
    models[oof_season] = xgb.train(
        params=param,
        dtrain=dtrain,
        num_boost_round = num_rounds,        
    )
    preds = models[oof_season].predict(dval)
    print(f"oof season {oof_season} mae: {mean_absolute_error(y_val, preds)}")
    oof_mae.append(mean_absolute_error(y_val, preds))
    oof_preds += list(preds)
    oof_targets += list(y_val)
    oof_ss += list(s_val)
print(f"average mae: {np.mean(oof_mae)}")

oof season 2005 mae: 8.164066108981045
oof season 2006 mae: 8.479049947794476
oof season 2007 mae: 7.91828537787854
oof season 2008 mae: 9.608090389222424
oof season 2009 mae: 9.399405187994448
oof season 2010 mae: 8.646759467641559
oof season 2011 mae: 9.671915453051607
oof season 2012 mae: 8.439394149930877
oof season 2013 mae: 9.728069122119082
oof season 2014 mae: 9.98869893941312
oof season 2015 mae: 7.968151155744607
oof season 2016 mae: 10.224463286106277
oof season 2017 mae: 9.52647076366135
oof season 2018 mae: 10.524568229799087
oof season 2019 mae: 9.298190732412516
oof season 2021 mae: 10.69130220400646
oof season 2022 mae: 10.416495068103174
oof season 2023 mae: 9.644173551616483
oof season 2024 mae: 9.55531540777145
average mae: 9.362782344381504


In [ ]:
df_2 = pd.DataFrame(
    {"Season": oof_ss, "pred": oof_preds, "label": [(t > 0) * 1 for t in oof_targets], "gender": df["gender"]}
)
df_2["pred_pointdiff"] = df_2["pred"].astype(int)

xdf_all = df_2.clip(-30, 30).groupby("pred_pointdiff")["label"].mean().reset_index(name="average_win_pct")
xdf_men = df_2.clip(-30, 30).loc[df["gender"] == 0].groupby("pred_pointdiff")["label"].mean().reset_index(name="average_win_pct")
xdf_women = df_2.clip(-30, 30).loc[df["gender"] == 1].groupby("pred_pointdiff")["label"].mean().reset_index(name="average_win_pct")

seaborn.lineplot(x=xdf_all["pred_pointdiff"], y=xdf_all["average_win_pct"])
seaborn.lineplot(x=xdf_men["pred_pointdiff"], y=xdf_men["average_win_pct"])
seaborn.lineplot(x=xdf_women["pred_pointdiff"], y=xdf_women["average_win_pct"])


In [ ]:
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
t = 25
dat = list(zip(oof_preds, np.array(oof_targets)>0))
dat = sorted(dat, key = lambda x: x[0])
pred, label = list(zip(*dat))
spline_model = UnivariateSpline(np.clip(pred, -t, t), label, k=5)
spline_fit = np.clip(spline_model(np.clip(oof_preds, -t, t)), 0.01, 0.99)
print(f"brier: {brier_score_loss(np.array(oof_targets)>0, spline_fit)}")
df_2["spline"] = spline_fit
xdf = df_2.clip(-30,30).groupby('pred_pointdiff')[['spline','label']].mean().reset_index()

plt.figure()
plt.plot(xdf['pred_pointdiff'],xdf['label'])
plt.plot(xdf['pred_pointdiff'],xdf['spline'])

In [ ]:
print(f"brier: {brier_score_loss(np.array(oof_targets)>0, spline_fit)}")

for oof_season in set(df.Season):
    x = df_2.loc[df["Season"] == oof_season, "spline"].values
    y = df_2.loc[df["Season"] == oof_season, "label"].values
    print(oof_season, np.round(brier_score_loss(y, x),5))

In [ ]:
X = pd.read_csv(f"{data_dir}\SampleSubmissionStage2.csv")

X['Season'] = X['ID'].apply(lambda t: int(t.split('_')[0]))
X['T1_TeamID'] = X['ID'].apply(lambda t: int(t.split('_')[1]))
X['T2_TeamID'] = X['ID'].apply(lambda t: int(t.split('_')[2]))
X['gender'] = X['T1_TeamID'].apply(lambda t: 0 if str(t)[0]=='1' else 1)
X = pd.merge(X,season_median_T1 , on = ['Season', 'T1_TeamID'], how = 'left')
X = pd.merge(X,season_median_T2 , on = ['Season', 'T2_TeamID'], how = 'left')
X = pd.merge(X, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
X = pd.merge(X, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
X["seed_diff"] = X["T1_seed"] - X["T2_seed"]
X = pd.merge(X, glm_quality_T1, on=["Season", "T1_TeamID"], how="left")
X = pd.merge(X, glm_quality_T2, on=["Season", "T2_TeamID"], how="left")
X["diff_quality"] = X["T1_quality"] - X["T2_quality"]
X = pd.merge(X, game_5_ratio_T1, on=["Season", "T1_TeamID"], how="left")
X = pd.merge(X, game_5_ratio_T2, on=["Season", "T2_TeamID"], how="left")
X["game_5_diff"] = X["T1_game_5_ratio"] - X["T2_game_5_ratio"]
X

In [ ]:
preds = []
for oof_season in set(df.Season):
    dtest = xgb.DMatrix(X[features].values)
    margin_preds = models[oof_season].predict(dtest) * 1.0 # aggressive submissions >1, conservative submissions <1
    probs = np.clip(spline_model(np.clip(margin_preds, -t, t)), 0.01, 0.99)
    preds.append(probs)
X['Pred'] = np.array(preds).mean(axis=0) 

In [ ]:
pd.pivot_table(data = X, index='T1_seed', columns='T2_seed', values='Pred', aggfunc='mean').style.bar(color='#5fba7d', vmin=0, vmax=1)

In [ ]:
prediction =X[['ID','Pred']].to_csv(f'{data_dir}\predictions_1.csv',index=None)